In [2]:
!pip install pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 76.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 68.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 20.8 MB/s eta 0:00:00


In [3]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import re


def extract_hash(string):
    match = re.search(r"FEATS_(.*?)_SWAPS", string)
    return match.group(1) if match else None

In [5]:
!beaker dataset fetch 01J686ACE7H0MH097V0KJKR7W1

Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 0          ⠙  
Bytes: 0 B        ⠙  
Files: 0          8 in progress ⠹  
Bytes: 0 B        300.7 MiB in progress ⠹  
Files: 0          8 in progress ⠸  
Bytes: 0 B        300.7 MiB in progress ⠸  
Files: 0          8 in progress ⠼  
Bytes: 0 B        300.7 MiB in progress ⠼  
Files: 0          8 in progress ⠴  
Bytes: 0 B        300.7 MiB in progress ⠴  
Files: 0          8 in progress ⠦  
Bytes: 0 B        300.7 MiB in progress ⠦  
Files: 0          8 in progress ⠧  
Bytes: 0 B        300.7 MiB in progress ⠧  
Files: 0          8 in progress ⠇  
Bytes: 0 B        300.7 MiB in progress ⠇  
Files: 1          8 in progress ⠏  
Bytes: 37.03 MiB  302.2 MiB in progress ⠏  
Files: 2          8 in progress ⠋  
Bytes: 74.64 MiB  302.1 MiB in progress ⠋  
Files: 5          8 in progress ⠙  
Bytes: 186.8 MiB  302.1 MiB in progress ⠙  
Files: 5          8 in progress ⠹  
Bytes: 186.8 MiB  302.1 MiB in progress ⠹  
Files: 5          8 in progress 

In [6]:
# FEATURES_PATH = Path("../../data/all_features_helpsteer2/data")
FEATURES_PATH = Path("data")
CSV_PATH = Path("test.csv")

In [7]:
filepaths = list(FEATURES_PATH.glob("*.jsonl"))
num_swaps = {}
for fp in tqdm(filepaths):
    df = pd.read_json(fp, lines=True)
    try:
        num_swaps[fp.name] = df["is_swapped"].value_counts()[True]
    except Exception:
        num_swaps[fp.name] = len(df)

swaps_df = (
    pd.DataFrame([num_swaps])
    .transpose()
    .rename(columns={0: "num_swaps"})
    .reset_index()
    .rename(columns={"index": "filename"})
)

100% 146/146 [00:42<00:00,  3.45it/s]


In [8]:
swaps_df["hash"] = swaps_df["filename"].apply(extract_hash)

In [13]:
swaps_df

In [10]:
result_df = pd.read_csv(CSV_PATH)
combined = result_df.merge(swaps_df, how="left", on="hash")

In [17]:
combined

,hash,model_type,chat_template,index,bertscore,bertscore_length,complexity_of_intents=complex,complexity_of_intents=moderate,complexity_of_intents=simple,cosine_sim,...,mt-bench-easy,mt-bench-hard,mt-bench-med,refusals-dangerous,refusals-offensive,xstest-should-refuse,xstest-should-respond,label,filename,num_swaps_y
0,6664fbc1d926adb447d65c676e5eda2d,Seq. Classifier,tulu,0,0,0,0,0,0,0,...,0.892857,0.648649,0.925,0.68,0.52,0.759740,0.880,1,human_datamodel_7000_FEATS_6664fbc1d926adb447d...,2907
1,9f5a414637204d94ec8e257d6099ead1,Seq. Classifier,tulu,46,0,0,0,0,0,0,...,0.857143,0.648649,0.925,0.52,0.69,0.766234,0.912,1,human_datamodel_7000_FEATS_9f5a414637204d94ec8...,6801
2,8264dd3241d6e8be8a420c73145b134e,Seq. Classifier,tulu,12,0,0,0,0,0,0,...,0.928571,0.675676,0.900,0.56,0.66,0.733766,0.892,1,human_datamodel_7000_FEATS_8264dd3241d6e8be8a4...,6808
3,017a74d4b8894de081aa4a34b528bd67,Seq. Classifier,tulu,55,0,0,0,0,1,0,...,0.892857,0.702703,0.850,0.47,0.57,0.811688,0.916,1,human_datamodel_7000_FEATS_017a74d4b8894de081a...,5077
4,6913ed04d8ab60bc1c32081045622cbd,Seq. Classifier,tulu,52,0,0,0,0,0,0,...,0.857143,0.594595,0.825,0.37,0.54,0.532468,0.916,1,human_datamodel_7000_FEATS_6913ed04d8ab60bc1c3...,6992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2893738f853a4d103c90b3a73f6bf4c8,Seq. Classifier,tulu,26,0,0,0,0,0,0,...,0.857143,0.621622,0.825,0.19,0.17,0.337662,0.876,1,human_datamodel_7000_FEATS_2893738f853a4d103c9...,82
66,01414f78b534bd2b158581a9e90bb978,Seq. Classifier,tulu,66,0,0,0,1,0,0,...,0.892857,0.594595,0.775,0.24,0.23,0.331169,0.876,0,human_datamodel_7000_FEATS_01414f78b534bd2b158...,390
67,7c8418ae17d6ca4b5c5c332685d31241,Seq. Classifier,tulu,7,0,0,0,0,1,0,...,0.892857,0.594595,0.825,0.19,0.13,0.331169,0.868,0,human_datamodel_7000_FEATS_7c8418ae17d6ca4b5c5...,13
68,78875c3aa32dbc58dbdd46096a9c877e,Seq. Classifier,tulu,42,0,0,0,0,0,0,...,0.892857,0.621622,0.850,0.19,0.14,0.311688,0.916,0,human_datamodel_7000_FEATS_78875c3aa32dbc58dbd...,413


In [18]:
combined.to_csv("combined.csv")